In [1]:
import re
import operator
import numpy as np
import seaborn as sns
import pandas as pd
import pickle as pickle #data processing
from collections import Counter #tokenization
import keras #ML
#import processing as pr #helper
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [2]:
shakespeare_db = pd.read_csv("Shakespeare_data.csv")

In [3]:
shakespeare_db

,Dataline,Play,PlayerLinenumber,ActSceneLine,Player,PlayerLine
0,1,Henry IV,NaN,NaN,NaN,ACT I
1,2,Henry IV,NaN,NaN,NaN,SCENE I. London. The palace.
2,3,Henry IV,NaN,NaN,NaN,"Enter KING HENRY, LORD JOHN OF LANCASTER, the ..."
3,4,Henry IV,1.0,1.1.1,KING HENRY IV,"So shaken as we are, so wan with care,"
4,5,Henry IV,1.0,1.1.2,KING HENRY IV,"Find we a time for frighted peace to pant,"
5,6,Henry IV,1.0,1.1.3,KING HENRY IV,And breathe short-winded accents of new broils
6,7,Henry IV,1.0,1.1.4,KING HENRY IV,To be commenced in strands afar remote.
7,8,Henry IV,1.0,1.1.5,KING HENRY IV,No more the thirsty entrance of this soil
8,9,Henry IV,1.0,1.1.6,KING HENRY IV,Shall daub her lips with her own children's bl...
9,10,Henry IV,1.0,1.1.7,KING HENRY IV,"Nor more shall trenching war channel her fields,"


In [4]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(shakespeare_db['PlayerLine'])

In [5]:
uniques = vectorizer.get_feature_names()

In [6]:
word_array = X.toarray()

In [7]:
def keywords(docs, k):
    tok = Tokenizer()
    tok.fit_on_texts(docs)
    tfidf_matrix = tok.texts_to_matrix(docs,mode='tfidf')
    tfidf_scores = np.sum(tfidf_matrix, axis=0)
    return sorted(dict(zip(tok.word_index,tfidf_scores)).items(),key=lambda x:x[1])

In [153]:
docs = ["The sky is blue sky sky", "I sky love blue the sky", "blue let's sky kill some seagulls"]

In [9]:
keywords(docs,5)

[('the', 0.0),
 ('blue', 0.9162907318741551),
 ('i', 0.9162907318741551),
 ('love', 0.9162907318741551),
 ("let's", 0.9162907318741551),
 ('kill', 0.9162907318741551),
 ('some', 0.9162907318741551),
 ('seagulls', 0.9162907318741551),
 ('sky', 1.3862943611198906),
 ('is', 1.3862943611198906)]

In [148]:
def alt_keywords(docs, k):
    lines = [line.rstrip() for line in open('SmartStoplist.txt')]
    stop_list = set(lines)
    vectorizer = TfidfVectorizer(stop_words=stop_list)
    tfidf_matrix = vectorizer.fit_transform(docs)
    tfidf_scores = np.sum(tfidf_matrix,axis=0)
    tfidf_scores = np.ravel(tfidf_scores)
    return sorted(dict(zip(vectorizer.get_feature_names(),tfidf_scores)).items(),key=lambda x:x[1], reverse=True)[:k]

In [154]:
alt_keywords(docs,2)

/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['1971', 'ain', 'aren', 'cornell', 'couldn', 'cs', 'didn', 'doesn', 'don', 'english', 'ftp', 'hadn', 'hasn', 'haven', 'isn', 'list', 'mon', 'pub', 'salton', 'shouldn', 'smart', 'stop', 've', 'wasn', 'weren', 'won', 'word', 'wouldn'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


[('sky', 2.021347396245477), ('blue', 1.0323566767442478)]

In [12]:
def get_vocab(lst): #tokenizing
    vocabcount, vocab = Counter(w for txt in lst for w in txt.split())
    return vocab, vocabcount


In [13]:
vocab, vocabcount = get_vocab(docs)

ValueError: too many values to unpack (expected 2)

In [14]:
path = 'glove.6b.zip'
glove_weights = get_glove_weights(path, origin="https://nlp.stanford.edu/data/glove.6b.zip")


NameError: name 'get_glove_weights' is not defined

In [15]:
FN = 'vocabulary-embedding'

In [16]:

seed=42


In [17]:
vocab_size = 40000

In [18]:

embedding_dim = 100


In [19]:

lower = False # dont lower case the text

In [20]:

import pickle as pickle
FN0 = 'tokens' # this is the name of the data file which I assume you already have
with open('data/%s.pkl'%FN0, 'rb') as fp:
    heads, desc, keywords = pickle.load(fp) # keywords are not used in this project

FileNotFoundError: [Errno 2] No such file or directory: 'data/tokens.pkl'

In [21]:
def histogram(words):
    #words_list = re.split("(?:(?:[^a-zA-Z]+')|(?:'[^a-zA-Z]+))|(?:[^a-zA-Z']+)", words)
    words_list = re.split("\W*[^\'\w+\']", words)
    #thanks to Martijn Pieters for the above split
    word_dictionary = {}
    counter = len(words_list)
    i = 0
    while i < counter:
        word_dictionary[words_list[i]] = 1
        n = i+1
        while n < counter:
            if words_list[i] == words_list[n]:
                word_dictionary[words_list[i]] += 1
                words_list.pop(n)
                counter -= 1
                n -= 1
            n += 1
        i += 1
    return word_dictionary
def unique_words(histogram):
    return len(histogram)
def frequency(word, histogram):
    return histogram[word]
def create_sorted_array(histogram):
    new_array = []
    count = 0
    for i in histogram:
        new_array.append([histogram[i]])
        new_array[count].append(i)
        count += 1
    new_array.sort()
    return new_array

In [22]:
hamlet = shakespeare_db[shakespeare_db['Play']=='Hamlet']

In [23]:
hamlet = hamlet['PlayerLine']

In [24]:
def play_string(play):
    string = ""
    for i in play:
        string += i
        string += " "
    return string

In [25]:
hamlet_string = play_string(hamlet)

In [26]:
hamlet_histogram = histogram(hamlet_string)

In [27]:
hamlet_list = create_sorted_array(hamlet_histogram)

In [28]:
hamlet_list[len(hamlet_list)-600]

[6, 'toward']

In [29]:
alt_keywords(hamlet,10)

/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['1971', 'ain', 'aren', 'cornell', 'couldn', 'cs', 'didn', 'doesn', 'don', 'english', 'ftp', 'hadn', 'hasn', 'haven', 'isn', 'list', 'll', 'mon', 'pub', 'salton', 'shouldn', 'smart', 'stop', 've', 'wasn', 'weren', 'won', 'word', 'wouldn'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


[('hamlet', 53.745814144803454),
 ('father', 33.12246310220759),
 ('ll', 32.02239901534809),
 ('love', 30.325235242783826),
 ('horatio', 26.678890010238646),
 ('man', 26.088849758745447),
 ('exeunt', 26.06427473312663),
 ('hath', 25.642089354834894),
 ('give', 25.300050959695444),
 ('laertes', 23.689480901559044)]

In [30]:
keywords(hamlet,10)

[('the', 0.0),
 ('relief', 7.660585461703256),
 ('stirring', 7.660585461703256),
 ('rivals', 7.660585461703256),
 ('liegemen', 7.660585461703256),
 ('holla', 7.660585461703256),
 ('belief', 7.660585461703256),
 ('dreaded', 7.660585461703256),
 ('entreated', 7.660585461703256),
 ('minutes', 7.660585461703256),
 ('assail', 7.660585461703256),
 ('fortified', 7.660585461703256),
 ('yond', 7.660585461703256),
 ('westward', 7.660585461703256),
 ('illume', 7.660585461703256),
 ('scholar', 7.660585461703256),
 ('harrows', 7.660585461703256),
 ("usurp'st", 7.660585461703256),
 ('stalks', 7.660585461703256),
 ('avouch', 7.660585461703256),
 ('combated', 7.660585461703256),
 ("frown'd", 7.660585461703256),
 ('angry', 7.660585461703256),
 ('parle', 7.660585461703256),
 ('smote', 7.660585461703256),
 ('sledded', 7.660585461703256),
 ('polacks', 7.660585461703256),
 ('martial', 7.660585461703256),
 ('stalk', 7.660585461703256),
 ('opinion', 7.660585461703256),
 ('bodes', 7.660585461703256),
 ('erupt

In [31]:
def find_alt_key_words(play_name, num):
    play = shakespeare_db[shakespeare_db['Play']==play_name]
    play = play['PlayerLine']
    alt_keys = alt_keywords(play,num)
    return alt_keys

In [32]:
def find_key_words(words):
    play = shakespeare_db[shakespeare_db['Play']==play_name]
    play = play['PlayerLine']
    keys = alt_keywords(play,5)
    return keys

In [145]:
find_alt_key_words('Hamlet', 5)

/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['1971', 'ain', 'aren', 'cornell', 'couldn', 'cs', 'didn', 'doesn', 'don', 'english', 'ftp', 'hadn', 'hasn', 'haven', 'isn', 'list', 'mon', 'pub', 'salton', 'shouldn', 'smart', 'stop', 've', 'wasn', 'weren', 'won', 'word', 'wouldn'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


[('hamlet', 55.71800525611245),
 ('father', 33.29970822958044),
 ('love', 30.90675374725298),
 ('horatio', 27.02397205251524),
 ('give', 25.52107822916803)]

In [34]:
find_alt_key_words('Henry V', 10)

/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['1971', 'ain', 'aren', 'cornell', 'couldn', 'cs', 'didn', 'doesn', 'don', 'english', 'ftp', 'hadn', 'hasn', 'haven', 'isn', 'list', 'll', 'mon', 'pub', 'salton', 'shouldn', 'smart', 'stop', 've', 'wasn', 'weren', 'won', 'word', 'wouldn'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


[('france', 37.37086360646235),
 ('god', 28.992458666275496),
 ('man', 24.941733372922936),
 ('french', 23.850141964261752),
 ('exeunt', 23.6281527825009),
 ('hath', 23.48928406721036),
 ('de', 23.247519142440783),
 ('england', 22.63581814364887),
 ('day', 22.29651900246219),
 ('majesty', 21.913216507932734)]

In [35]:
def all_the_plays():
    new_play = "Henry IV"
    play_array = []
    for i in shakespeare_db['Play']:
        if i != new_play:
            play_array.append(new_play)
            new_play = i
    return play_array

In [36]:
all_plays = all_the_plays()

In [37]:
play_keywords = [[]]   

In [71]:
play_groups = []

In [155]:
def find_keys():
    for i in all_plays:
        play_words = []
        play_words.append(i)
        play_words.append(find_alt_key_words(i, 10))
        play_keywords.append(play_words)

In [160]:
find_keys()

/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['1971', 'ain', 'aren', 'cornell', 'couldn', 'cs', 'didn', 'doesn', 'don', 'english', 'ftp', 'hadn', 'hasn', 'haven', 'isn', 'list', 'mon', 'pub', 'salton', 'shouldn', 'smart', 'stop', 've', 'wasn', 'weren', 'won', 'word', 'wouldn'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['1971', 'ain', 'aren', 'cornell', 'couldn', 'cs', 'didn', 'doesn', 'don', 'english', 'ftp', 'hadn', 'hasn', 'haven', 'isn', 'list', 'mon', 'pub', 'salton', 'shouldn', 'smart', 'stop', 've', 'wasn', 'weren', 'won', 'word', 'wouldn'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/anaconda3/lib/python3.7

/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['1971', 'ain', 'aren', 'cornell', 'couldn', 'cs', 'didn', 'doesn', 'don', 'english', 'ftp', 'hadn', 'hasn', 'haven', 'isn', 'list', 'mon', 'pub', 'salton', 'shouldn', 'smart', 'stop', 've', 'wasn', 'weren', 'won', 'word', 'wouldn'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['1971', 'ain', 'aren', 'cornell', 'couldn', 'cs', 'didn', 'doesn', 'don', 'english', 'ftp', 'hadn', 'hasn', 'haven', 'isn', 'list', 'mon', 'pub', 'salton', 'shouldn', 'smart', 'stop', 've', 'wasn', 'weren', 'won', 'word', 'wouldn'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/anaconda3/lib/python3.7

In [161]:
play_keywords

[['Henry IV',
  [('prince', 24.749750330310427),
   ('art', 24.337388923164305),
   ('god', 22.35880717849191),
   ('percy', 20.976693911557774),
   ('hear', 20.09403877956212),
   ('time', 19.9314285443733),
   ('falstaff', 19.13343463126367),
   ('douglas', 18.828825567735954),
   ('harry', 18.758553537274214),
   ('john', 18.721756515325836)]],
 ['Henry VI Part 1',
  [('talbot', 39.692646548986076),
   ('richard', 35.57881685461415),
   ('bishop', 27.736297271171686),
   ('france', 26.146804067636758),
   ('henry', 22.894460031003714),
   ('lords', 18.15240828950604),
   ('death', 17.686660698786945),
   ('father', 17.538113029999895),
   ('orleans', 15.55724607389372),
   ('duke', 15.436343781268699)]],
 ['Henry VI Part 2',
  [('duke', 37.18218680595654),
   ('suffolk', 34.492417266267964),
   ('york', 31.736326742654242),
   ('gloucester', 25.067559232801077),
   ('death', 24.736830200920913),
   ('god', 24.694280012935693),
   ('henry', 23.107227371939214),
   ('humphrey', 22.300

In [87]:
def find_groups(keys):
    for i in keys:
        play_thing = [[]]
        for n in i:
            if (play_thing == None) and (i[0] != None):
                play_thing.append([i[0]])
            if n != i[0]:
                for y in n:
                    for x in keys:
                        play_match_words = []
                        for xn in x:
                            if xn != n:
                                if xn != x[0]:
                                    for xy in xn:
                                        if xy[0] == y[0]:
                                            play_match_words.append(xy[0])
                            if (play_match_words != None):
                                play_match = [[x[0]]]
                                play_match[0].append(play_match_words)
                                play_thing[0].append(play_match)
        play_groups.append(play_thing)
                                            

In [88]:
find_groups(play_keywords)

In [89]:
play_groups

[[],
 [[]],
 [[[['Henry IV', []]],
   [['Henry IV', []]],
   [['Henry VI Part 1', ['ll']]],
   [['Henry VI Part 1', ['ll']]],
   [['Henry VI Part 2', ['ll']]],
   [['Henry VI Part 2', ['ll']]],
   [['Henry VI Part 3', ['ll']]],
   [['Henry VI Part 3', ['ll']]],
   [['Alls well that ends well', ['ll']]],
   [['Alls well that ends well', ['ll']]],
   [['As you like it', ['ll']]],
   [['As you like it', ['ll']]],
   [['Antony and Cleopatra', ['ll']]],
   [['Antony and Cleopatra', ['ll']]],
   [['A Comedy of Errors', ['ll']]],
   [['A Comedy of Errors', ['ll']]],
   [['Coriolanus', ['ll']]],
   [['Coriolanus', ['ll']]],
   [['Cymbeline', ['ll']]],
   [['Cymbeline', ['ll']]],
   [['Hamlet', ['ll']]],
   [['Hamlet', ['ll']]],
   [['Henry V', []]],
   [['Henry V', []]],
   [['Henry VIII', ['ll']]],
   [['Henry VIII', ['ll']]],
   [['King John', []]],
   [['King John', []]],
   [['Julius Caesar', []]],
   [['Julius Caesar', []]],
   [['King Lear', ['ll']]],
   [['King Lear', ['ll']]],
   [['Lo

In [139]:
def word_groups(keys):
    word_array = []
    for i in keys:
        for n in i:
            if n != i[0]:
                for y in n:
                    word_in_array = False
                    for w in word_array:
                        if w[0] == y[0]:
                            w.append(i[0])
                            word_in_array = True
                    if word_in_array != True:
                        single_word = [y[0]]
                        single_word.append(i[0])
                        word_array.append(single_word)
    return word_array

In [162]:
w = word_groups(play_keywords)

In [163]:
w

[['prince', 'Henry IV', 'Much Ado about nothing'],
 ['art',
  'Henry IV',
  'King Lear',
  'Romeo and Juliet',
  'The Tempest',
  'Timon of Athens'],
 ['god',
  'Henry IV',
  'Henry VI Part 2',
  'Henry V',
  'Much Ado about nothing',
  'Richard II',
  'Richard III'],
 ['percy', 'Henry IV'],
 ['hear',
  'Henry IV',
  'Antony and Cleopatra',
  'Coriolanus',
  'Much Ado about nothing'],
 ['time',
  'Henry IV',
  'As you like it',
  'A Comedy of Errors',
  'Cymbeline',
  'macbeth',
  'The Tempest'],
 ['falstaff', 'Henry IV', 'Merry Wives of Windsor'],
 ['douglas', 'Henry IV'],
 ['harry', 'Henry IV'],
 ['john', 'Henry IV', 'Merry Wives of Windsor'],
 ['talbot', 'Henry VI Part 1'],
 ['richard', 'Henry VI Part 1', 'Richard II', 'Richard III'],
 ['bishop', 'Henry VI Part 1'],
 ['france', 'Henry VI Part 1', 'Henry V', 'King John'],
 ['henry', 'Henry VI Part 1', 'Henry VI Part 2', 'Henry VI Part 3'],
 ['lords', 'Henry VI Part 1'],
 ['death',
  'Henry VI Part 1',
  'Henry VI Part 2',
  'Richard 

In [178]:
def play_groups(keys):
    play_array = []
    for i in keys:
        array_within_array = []
        for n in i:
            if n == i[0]:
                array_within_array.append(n)
            else:
                for y in n:
                    for z in w:
                        if y[0] == z[0]:
                            for a in z:
                                repeat = False
                                for b in array_within_array:
                                    if a == b[0]:
                                        repeat = True
                                        b[1] += 1
                                if (repeat != True) and (a != y[0]) and (a != i[0]):
                                    array_within_array.append([a,1])
        play_array.append(array_within_array)
        
    return play_array

In [179]:
p = play_groups(play_keywords)

In [180]:
p

[['Henry IV',
  ['Much Ado about nothing', 3],
  ['King Lear', 1],
  ['Romeo and Juliet', 1],
  ['The Tempest', 2],
  ['Timon of Athens', 1],
  ['Henry VI Part 2', 1],
  ['Henry V', 1],
  ['Richard II', 1],
  ['Richard III', 1],
  ['Antony and Cleopatra', 1],
  ['Coriolanus', 1],
  ['As you like it', 1],
  ['A Comedy of Errors', 1],
  ['Cymbeline', 1],
  ['macbeth', 1],
  ['Merry Wives of Windsor', 2]],
 ['Henry VI Part 1',
  ['Richard II', 3],
  ['Richard III', 2],
  ['Henry V', 1],
  ['King John', 2],
  ['Henry VI Part 2', 3],
  ['Henry VI Part 3', 2],
  ['Romeo and Juliet', 1],
  ['As you like it', 2],
  ['Cymbeline', 1],
  ['Hamlet', 1],
  ['King Lear', 1],
  ['Pericles', 1],
  ['Taming of the Shrew', 1],
  ['The Tempest', 1],
  ['Measure for measure', 1]],
 ['Henry VI Part 2',
  ['Henry VI Part 1', 3],
  ['As you like it', 1],
  ['Measure for measure', 1],
  ['Richard II', 3],
  ['Henry VI Part 3', 2],
  ['King Lear', 1],
  ['Richard III', 2],
  ['Romeo and Juliet', 1],
  ['Henry 